> library dependencies

+ itunes API searcher
    + ```$ pip install pyitunes```

In [1]:
import itunes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bs4 as bs
import urllib.request

### this is a wrapper for the [iTunes Search Api]()

In [2]:
# for example
item = itunes.search (query='star wars')
print (item)

[<Feature-Movie>: Star Wars: The Force Awakens, <Feature-Movie>: Star Wars: The Phantom Menace, <Feature-Movie>: Star Wars: The Empire Strikes Back, <Tv-Episode>: R2 Come Home, <Feature-Movie>: Star Wars: Return of the Jedi, <Feature-Movie>: Star Wars: A New Hope, <Feature-Movie>: Star Wars: Attack of the Clones, <Feature-Movie>: Star Wars: Revenge of the Sith, <Podcast>: Rebel Force Radio: Star Wars Podcast, <Tv-Episode>: Bombad Jedi, <Tv-Episode>: An Old Friend, <Tv-Episode>: Weapons Factory, <Tv-Episode>: The Lawless, <Tv-Episode>: Citadel Rescue, <Song>: Star Wars, <Tv-Episode>: Point of No Return, <Tv-Episode>: Bounty, <Tv-Episode>: Ambush, <Tv-Episode>: The Box, <Tv-Episode>: Rising Malevolence, <Tv-Episode>: The Unknown, <Tv-Episode>: Rookies, <Tv-Episode>: Sacrifice, <Tv-Episode>: Defenders of Peace, <Tv-Episode>: Downfall of a Droid, <Tv-Episode>: Conspiracy, <Tv-Episode>: Mystery of the Thousand Moons, <Tv-Episode>: Shadow of Malevolence, <Tv-Episode>: Hostage Crisis]


## This searches for podcasts, instead of music albums,
### and prroduces

### Set the query term and Run All Cells
### ⇩


In [3]:
# queryterm = 'cystic fibrosis'
queryterm = 'infectious disease'
# queryterm = 'genetic'
# queryterm = 'covid19'
# queryterm = 'foreclosure'
podsearch = itunes.search(query=queryterm, media='podcast')

In [4]:
for p in podsearch:
    print (p)

<Podcast>: Infectious Diseases Society of America
<Podcast>: Emerging Infectious Diseases
<Podcast>: Persiflagers Infectious Disease Puscast
<Podcast>: Persiflagers Infectious Disease Puscast
<Podcast>: Breakpoints
<Podcast>: Infectious Disease
<Podcast>: The Open Forum Infectious Diseases Podcast
<Podcast>: Infectious Disease & Antibiotic Resistance
<Podcast>: PeerView Infectious Diseases CME/CNE/CPE Audio Podcast
<Podcast>: Infectious Disease Dynamics
<Podcast>: Disease State - Infectious
<Podcast>: PeerVoice Immunology & Infectious Disease Video
<Podcast>: Infectious Questions : An Infectious Diseases Public Health Podcast
<Podcast>: Infectious Diseases
<Podcast>: Podcasts – PKIDs Blog
<Podcast>: Infectious Diseases and Pandemics (video)
<Podcast>: Infectious Diseases and Pandemics (audio)
<Podcast>: PeerVoice Immunology & Infectious Disease Audio
<Podcast>: PeerView Infectious Diseases CME/CNE/CPE Video Podcast
<Podcast>: The History of Infectious Diseases


In [5]:
# let's looks at what gets returned for each item in that list of Podcast search results
print (vars(podsearch[0]))

{'id': 519582740, 'name': 'Infectious Diseases Society of America', 'url': 'https://podcasts.apple.com/us/podcast/infectious-diseases-society-of-america/id519582740?uo=4', '_release_date': None, 'artwork': {'30': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/30x30bb.jpg', '60': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/60x60bb.jpg', '600': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/600x600bb.jpg'}, 'json': {'wrapperType': 'track', 'kind': 'podcast', 'collectionId': 519582740, 'trackId': 519582740, 'artistName': 'IDSA', 'collectionName': 'Infectious Diseases Society of America', 'trackName': 'Infectious Diseases Society of America', 'collectionCensoredName': 'Infectious Diseases Society of America', 'trackCensoredName': 'Infectious Diseas

### ⇪ A little wierd, looks like most of what we want is packed into
### a "JSON" attribute, so...there it is.

- - -

# So let's lookup some stuff about the podcast in question

+ no. of individual posts / episodes per year since publishing
+ length posts (minutes)
+ rating per channel +/or track, if available

### first for # posts / episodes / media URL

In [6]:
print (f'{len(podsearch)} channels\n')
for channel in podsearch:
    print (f'{channel.json["trackCount"]} podcasts in "{channel.json["collectionName"]}" (with id = {channel.id})')
    print (f'The last post in this channel was {channel.json["releaseDate"]}, and iTunes page lising all podcast episodes:')
    print (f'{channel.json["collectionViewUrl"]}\n')

20 channels

55 podcasts in "Infectious Diseases Society of America" (with id = 519582740)
The last post in this channel was 2020-05-26T15:59:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/infectious-diseases-society-of-america/id519582740?uo=4

50 podcasts in "Emerging Infectious Diseases" (with id = 212828612)
The last post in this channel was 2020-05-21T15:56:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/emerging-infectious-diseases/id212828612?uo=4

300 podcasts in "Persiflagers Infectious Disease Puscast" (with id = 79930757)
The last post in this channel was 2020-05-17T08:25:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/persiflagers-infectious-disease-puscast/id79930757?uo=4

300 podcasts in "Persiflagers Infectious Disease Puscast" (with id = 1153935086)
The last post in this channel was 2020-05-17T08:25:00Z, and iTunes page lising all podcast episodes:
ht

## Fine.
### Uinfortunately, that 'iTunes Collection Preview' page only lists the first few episodes for the channel
### But podcast publisher's feed (not Apple) includes ALL podcasts 

In [7]:
channel_number = []
channel_name = []
channel_episodecount = []
channel_itunespreview = []
channel_feed = []

In [8]:
# "feedUrl" is not consistenly present, so handle missing Attributes

# definition to make write podcast channels to an array
def writeKeys(channel):
    channel_number.append(channel.id)
    channel_name.append(channel.json["collectionName"])
    channel_episodecount.append(channel.json["trackCount"])
    channel_itunespreview.append(channel.json["collectionViewUrl"])
    # feedURL is not consistent, check if present
    try:
       channel_feed.append(channel.json["feedUrl"])
    except KeyError as e:
        print (f'{e} does not exist in this channel')
        channel_feed.append(None)

In [9]:
for channel in podsearch:
    writeKeys(channel)

'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel


In [10]:
len(channel_name)

20

In [11]:
# create an "Channels" dataframe
data = {'ID': channel_number,
        'CHANNEL_NAME': channel_name,
        'EPISODE_COUNT': channel_episodecount,
        'ITUNES_PREVIEW_URL': channel_itunespreview,
        'PUBLISHER_FEED': channel_feed
       }
channels = pd.DataFrame(data,columns=['ID','CHANNEL_NAME','EPISODE_COUNT','ITUNES_PREVIEW_URL','PUBLISHER_FEED'])
channels

,ID,CHANNEL_NAME,EPISODE_COUNT,ITUNES_PREVIEW_URL,PUBLISHER_FEED
0,519582740,Infectious Diseases Society of America,55,https://podcasts.apple.com/us/podcast/infectio...,https://idsocietypodcast.libsyn.com/rss
1,212828612,Emerging Infectious Diseases,50,https://podcasts.apple.com/us/podcast/emerging...,https://www2c.cdc.gov/podcasts/createrss.asp?t...
2,79930757,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
3,1153935086,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
4,1470308447,Breakpoints,23,https://podcasts.apple.com/us/podcast/breakpoi...,https://pinecast.com/feed/sidp
5,415333788,Infectious Disease,49,https://podcasts.apple.com/us/podcast/infectio...,http://wwwf.imperial.ac.uk/imedia/itunes_colle...
6,1368469324,The Open Forum Infectious Diseases Podcast,28,https://podcasts.apple.com/us/podcast/the-open...,https://feeds.blubrry.com/feeds/the_ofid.xml
7,621579678,Infectious Disease & Antibiotic Resistance,7,https://podcasts.apple.com/us/podcast/infectio...,None
8,275853748,PeerView Infectious Diseases CME/CNE/CPE Audio...,16,https://podcasts.apple.com/us/podcast/peerview...,https://c.peerview.com/podcast/xml/infectiousd...
9,833385368,Infectious Disease Dynamics,53,https://podcasts.apple.com/us/podcast/infectio...,http://rss.sms.cam.ac.uk/rss/collection/153939...


In [12]:
channels.PUBLISHER_FEED[0]

'https://idsocietypodcast.libsyn.com/rss'

## Next: create an "Episodes" dataframe
### Constructed by keys in this 'dict'-like feed of each individual podcast episode

In [13]:
# what are the fields?
n = 6
request = urllib.request.Request(channels.PUBLISHER_FEED[n],headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'})
source = urllib.request.urlopen(request)

# this is an XML feel
# inspection reveals that each podcast is encapsulated in <item> tags
soup = bs.BeautifulSoup(source,'lxml')
rows = soup.find_all('item')
print (f'"{channels.CHANNEL_NAME[n]}" channel has {len(rows)} episodes: here is the {n}th episode from that feed:\n')
print (rows[n])

"The Open Forum Infectious Diseases Podcast" channel has 28 episodes: here is the 6th episode from that feed:

<item>
<title>An interview with Dr. Eric Winer</title>
<link/>https://www.blubrry.com/the_ofid/44509529/an-interview-with-dr-eric-winer/
      <guid>http://www.blubrry.com/the_ofid/44509529/an-interview-with-dr-eric-winer/</guid>
<dc:creator>Oxford University Press</dc:creator>
<pubdate>Mon, 12 Nov 2018 08:24:00 -0500</pubdate>
<description>&lt;p&gt;In this episode, OFID Editor Paul Sax, MD, interviews Eric Winer, MD, about his enduring battle with hemophilia. Diagnosed at 13 months old, he shares how early exposure to hospitals and doctors ignited his drive to become a physician, and how his subsequent diagnoses with HIV, hepatitis C, and numerous treatment complications did not deter his work resolve or his determination to have a family – and an extraordinarily productive career.&lt;/p&gt;</description>
<content:encoded><span>In this episode, </span><em>OFID</em><span> Edit

In [14]:
for r in rows:
    per = r.find_all("itunes:duration")
    print (per)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [15]:
# df = pd.DataFrame(columns = ['ID','CHANNEL_NAME','EPISODE_TITLE','DATE_PUBLISHED','EPISODE_LENGTH','MEDIA_URL'])

In [16]:
channel_ids = []
channel_names = []
episode_titles = []
episode_published = []
episode_length = []
episode_mediaurls = []

In [17]:
def getEpisodeList(url):
    #since it's a request from the messy web, let's add some error handling
    try:
        request = urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'})
        source = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print (f'Failed {e}: {url}')
        return None
    try:
        soup = bs.BeautifulSoup(source,'lxml')
        episodes = soup.find_all('item')
    except AttributeError as e:
        return None
    return episodes

In [18]:
def writeEpisode(row):
    if (row["PUBLISHER_FEED"]!=None):
#         print(row["ID"])
        #this is where you extract and append the individual Episode keys
        #repeating these two lines for each episode added to array
        r = getEpisodeList(row["PUBLISHER_FEED"])
        print (f'This podcast has {len(r)} episodes')
        for x in r:
            episode_titles.append(x.text)
            channel_ids.append(row["ID"])
            channel_names.append(row["CHANNEL_NAME"])
            try:
                length = x.find("itunes:duration")
                episode_length.append(length.text)
            except:
                episode_length.append(None)
            try:
                published_on = x.pubdate
                episode_published.append(published_on.text)
            except:
                episode_published.append(published_on.text)
channels.apply(writeEpisode, axis = 1)

This podcast has 55 episodes
This podcast has 50 episodes
This podcast has 323 episodes
This podcast has 323 episodes
This podcast has 23 episodes
This podcast has 49 episodes
This podcast has 28 episodes
This podcast has 16 episodes
This podcast has 53 episodes
This podcast has 17 episodes
This podcast has 25 episodes
This podcast has 26 episodes
This podcast has 8 episodes
This podcast has 10 episodes
This podcast has 25 episodes
This podcast has 15 episodes


0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object

In [19]:
data = {'ID': channel_ids,
        'CHANNEL_NAME': channel_names,
        'EPISODE_TITLE': episode_titles,
        'EPISODE_LENGTH': episode_length,
        'EPISODE_PUBLISHED': episode_published
       }
episodes = pd.DataFrame(data, columns =['ID','CHANNEL_NAME','EPISODE_TITLE','EPISODE_LENGTH','EPISODE_PUBLISHED'])
episodes

,ID,CHANNEL_NAME,EPISODE_TITLE,EPISODE_LENGTH,EPISODE_PUBLISHED
0,519582740,Infectious Diseases Society of America,\nCoronavirus: Clinical Trials & Registries (M...,25:42,"Tue, 26 May 2020 15:59:40 +0000"
1,519582740,Infectious Diseases Society of America,"\nCoronavirus: Educating ID Learners (May 23, ...",33:47,"Sat, 23 May 2020 07:30:00 +0000"
2,519582740,Infectious Diseases Society of America,\nCoronavirus: Strategies for Safely Reopening...,17:31,"Tue, 19 May 2020 09:00:00 +0000"
3,519582740,Infectious Diseases Society of America,"\nCoronavirus: Secondary Infections (May 16, 2...",20:54,"Sat, 16 May 2020 06:00:00 +0000"
4,519582740,Infectious Diseases Society of America,"\nCoronavirus: The Disparate Impact (May 12, 2...",28:00,"Tue, 12 May 2020 10:00:00 +0000"
...,...,...,...,...,...
1041,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRoy F. Chemaly, MD, MPH, FIDSA, FACP - Manag...",00:49:11,"Wed, 3 Apr 2019 18:00:00 -0500"
1042,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRichard S. Finn, MD - Breaking the Paradox: ...",01:15:38,"Fri, 8 Mar 2019 18:00:00 -0500"
1043,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nGregory Piazza, MD, MS - Turning the Tide on...",00:33:46,"Fri, 22 Feb 2019 18:00:00 -0500"
1044,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nMark S. Sulkowski, MD - Addressing the Evolv...",01:23:36,"Mon, 14 Jan 2019 18:00:00 -0500"


In [20]:
def totalMinutes(row):
    # A lot of the podcasts are missing durations! (..strange...)
    if not (row is None):
        # if duration does not list hours
        if (len(row.split(':')))==2:
            minutes, seconds = row.split(':')
            total_min = int(minutes)
            return total_min
        # if 'hours' are listed
        elif (len(row.split(':')))==3:
            hours, minutes, seconds = row.split(':')
            total_min = int(hours)*60 + int(minutes)
            return total_min
    else:
        return None

In [21]:
episodes["EPISODE_TOTAL_MINUTES"] = episodes["EPISODE_LENGTH"].apply(totalMinutes)

In [22]:
episodes

,ID,CHANNEL_NAME,EPISODE_TITLE,EPISODE_LENGTH,EPISODE_PUBLISHED,EPISODE_TOTAL_MINUTES
0,519582740,Infectious Diseases Society of America,\nCoronavirus: Clinical Trials & Registries (M...,25:42,"Tue, 26 May 2020 15:59:40 +0000",25.0
1,519582740,Infectious Diseases Society of America,"\nCoronavirus: Educating ID Learners (May 23, ...",33:47,"Sat, 23 May 2020 07:30:00 +0000",33.0
2,519582740,Infectious Diseases Society of America,\nCoronavirus: Strategies for Safely Reopening...,17:31,"Tue, 19 May 2020 09:00:00 +0000",17.0
3,519582740,Infectious Diseases Society of America,"\nCoronavirus: Secondary Infections (May 16, 2...",20:54,"Sat, 16 May 2020 06:00:00 +0000",20.0
4,519582740,Infectious Diseases Society of America,"\nCoronavirus: The Disparate Impact (May 12, 2...",28:00,"Tue, 12 May 2020 10:00:00 +0000",28.0
...,...,...,...,...,...,...
1041,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRoy F. Chemaly, MD, MPH, FIDSA, FACP - Manag...",00:49:11,"Wed, 3 Apr 2019 18:00:00 -0500",49.0
1042,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRichard S. Finn, MD - Breaking the Paradox: ...",01:15:38,"Fri, 8 Mar 2019 18:00:00 -0500",75.0
1043,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nGregory Piazza, MD, MS - Turning the Tide on...",00:33:46,"Fri, 22 Feb 2019 18:00:00 -0500",33.0
1044,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nMark S. Sulkowski, MD - Addressing the Evolv...",01:23:36,"Mon, 14 Jan 2019 18:00:00 -0500",83.0


In [23]:
import datetime

In [24]:
foo = "Sat, 23 May 2020 07:30:00 +0000"

In [25]:
anyblanks = episodes[episodes["EPISODE_PUBLISHED"]!=None]
anyblanks.shape

(1046, 6)

In [26]:
episodes

,ID,CHANNEL_NAME,EPISODE_TITLE,EPISODE_LENGTH,EPISODE_PUBLISHED,EPISODE_TOTAL_MINUTES
0,519582740,Infectious Diseases Society of America,\nCoronavirus: Clinical Trials & Registries (M...,25:42,"Tue, 26 May 2020 15:59:40 +0000",25.0
1,519582740,Infectious Diseases Society of America,"\nCoronavirus: Educating ID Learners (May 23, ...",33:47,"Sat, 23 May 2020 07:30:00 +0000",33.0
2,519582740,Infectious Diseases Society of America,\nCoronavirus: Strategies for Safely Reopening...,17:31,"Tue, 19 May 2020 09:00:00 +0000",17.0
3,519582740,Infectious Diseases Society of America,"\nCoronavirus: Secondary Infections (May 16, 2...",20:54,"Sat, 16 May 2020 06:00:00 +0000",20.0
4,519582740,Infectious Diseases Society of America,"\nCoronavirus: The Disparate Impact (May 12, 2...",28:00,"Tue, 12 May 2020 10:00:00 +0000",28.0
...,...,...,...,...,...,...
1041,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRoy F. Chemaly, MD, MPH, FIDSA, FACP - Manag...",00:49:11,"Wed, 3 Apr 2019 18:00:00 -0500",49.0
1042,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRichard S. Finn, MD - Breaking the Paradox: ...",01:15:38,"Fri, 8 Mar 2019 18:00:00 -0500",75.0
1043,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nGregory Piazza, MD, MS - Turning the Tide on...",00:33:46,"Fri, 22 Feb 2019 18:00:00 -0500",33.0
1044,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nMark S. Sulkowski, MD - Addressing the Evolv...",01:23:36,"Mon, 14 Jan 2019 18:00:00 -0500",83.0


In [27]:
episodes['EPISODE_YEAR'] = pd.to_datetime(episodes["EPISODE_PUBLISHED"], utc=True).dt.to_period('Y')
episodes

/home/xavier/anaconda3/lib/python3.7/site-packages/dateutil/parser/_parser.py:1218: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/home/xavier/anaconda3/lib/python3.7/site-packages/pandas/core/arrays/datetimes.py:1102: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


,ID,CHANNEL_NAME,EPISODE_TITLE,EPISODE_LENGTH,EPISODE_PUBLISHED,EPISODE_TOTAL_MINUTES,EPISODE_YEAR
0,519582740,Infectious Diseases Society of America,\nCoronavirus: Clinical Trials & Registries (M...,25:42,"Tue, 26 May 2020 15:59:40 +0000",25.0,2020
1,519582740,Infectious Diseases Society of America,"\nCoronavirus: Educating ID Learners (May 23, ...",33:47,"Sat, 23 May 2020 07:30:00 +0000",33.0,2020
2,519582740,Infectious Diseases Society of America,\nCoronavirus: Strategies for Safely Reopening...,17:31,"Tue, 19 May 2020 09:00:00 +0000",17.0,2020
3,519582740,Infectious Diseases Society of America,"\nCoronavirus: Secondary Infections (May 16, 2...",20:54,"Sat, 16 May 2020 06:00:00 +0000",20.0,2020
4,519582740,Infectious Diseases Society of America,"\nCoronavirus: The Disparate Impact (May 12, 2...",28:00,"Tue, 12 May 2020 10:00:00 +0000",28.0,2020
...,...,...,...,...,...,...,...
1041,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRoy F. Chemaly, MD, MPH, FIDSA, FACP - Manag...",00:49:11,"Wed, 3 Apr 2019 18:00:00 -0500",49.0,2019
1042,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRichard S. Finn, MD - Breaking the Paradox: ...",01:15:38,"Fri, 8 Mar 2019 18:00:00 -0500",75.0,2019
1043,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nGregory Piazza, MD, MS - Turning the Tide on...",00:33:46,"Fri, 22 Feb 2019 18:00:00 -0500",33.0,2019
1044,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nMark S. Sulkowski, MD - Addressing the Evolv...",01:23:36,"Mon, 14 Jan 2019 18:00:00 -0500",83.0,2019


In [28]:
# Lot of NaN wierdness in this list of minutes...because so many missing 'Durations'
minutes = set(episodes.EPISODE_TOTAL_MINUTES[np.logical_not(np.isnan(episodes.EPISODE_TOTAL_MINUTES))])
minutes

{0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 55.0,
 57.0,
 58.0,
 59.0,
 61.0,
 65.0,
 66.0,
 73.0,
 74.0,
 75.0,
 83.0,
 84.0,
 90.0,
 92.0}

In [29]:
MinutesIndex = pd.Index(episodes.EPISODE_TOTAL_MINUTES[np.logical_not(np.isnan(episodes.EPISODE_TOTAL_MINUTES))])
YearsIndex = pd.Index(episodes.EPISODE_YEAR)

In [30]:
MinutesIndex.name

'EPISODE_TOTAL_MINUTES'

In [31]:
YearsIndex

PeriodIndex(['2020', '2020', '2020', '2020', '2020', '2020', '2020', '2020',
             '2020', '2020',
             ...
             '2019', '2019', '2019', '2019', '2019', '2019', '2019', '2019',
             '2019', '2019'],
            dtype='period[A-DEC]', name='EPISODE_YEAR', length=1046, freq='A-DEC')

In [32]:
minutesaudit = pd.DataFrame(index=set(MinutesIndex))
yearsaudit = pd.DataFrame(index=set(YearsIndex))

In [33]:
minutesaudit.index.name = "MINUTES_RANGE"
yearsaudit.index.name = "YEARS_RANGE"

In [34]:
minutesaudit

""
MINUTES_RANGE
0.0
1.0
2.0
3.0
4.0
...
75.0
83.0
84.0


In [35]:
yearsaudit

""
YEARS_RANGE
2008
2019
2018
2013
2011
2020
2017
2010
2015


In [36]:
episodes.EPISODE_YEAR.min().year

2007

In [37]:
episodes.EPISODE_YEAR.max().year

2020

In [38]:
channels

,ID,CHANNEL_NAME,EPISODE_COUNT,ITUNES_PREVIEW_URL,PUBLISHER_FEED
0,519582740,Infectious Diseases Society of America,55,https://podcasts.apple.com/us/podcast/infectio...,https://idsocietypodcast.libsyn.com/rss
1,212828612,Emerging Infectious Diseases,50,https://podcasts.apple.com/us/podcast/emerging...,https://www2c.cdc.gov/podcasts/createrss.asp?t...
2,79930757,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
3,1153935086,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
4,1470308447,Breakpoints,23,https://podcasts.apple.com/us/podcast/breakpoi...,https://pinecast.com/feed/sidp
5,415333788,Infectious Disease,49,https://podcasts.apple.com/us/podcast/infectio...,http://wwwf.imperial.ac.uk/imedia/itunes_colle...
6,1368469324,The Open Forum Infectious Diseases Podcast,28,https://podcasts.apple.com/us/podcast/the-open...,https://feeds.blubrry.com/feeds/the_ofid.xml
7,621579678,Infectious Disease & Antibiotic Resistance,7,https://podcasts.apple.com/us/podcast/infectio...,None
8,275853748,PeerView Infectious Diseases CME/CNE/CPE Audio...,16,https://podcasts.apple.com/us/podcast/peerview...,https://c.peerview.com/podcast/xml/infectiousd...
9,833385368,Infectious Disease Dynamics,53,https://podcasts.apple.com/us/podcast/infectio...,http://rss.sms.cam.ac.uk/rss/collection/153939...


In [79]:
len(set(episodes.ID))

16

In [82]:
channels.dtypes

ID                     int64
CHANNEL_NAME          object
EPISODE_COUNT          int64
ITUNES_PREVIEW_URL    object
PUBLISHER_FEED        object
dtype: object

In [83]:
episodes.dtypes

ID                               int64
CHANNEL_NAME                    object
EPISODE_TITLE                   object
EPISODE_LENGTH                  object
EPISODE_PUBLISHED               object
EPISODE_TOTAL_MINUTES          float64
EPISODE_YEAR             period[A-DEC]
dtype: object

In [93]:
first = []
last = []
def defineYearRange(dfone,dftwo):
    filtered = dfone[dfone.PUBLISHER_FEED.notnull()]
    print (f'CHANNELS has {filtered.shape[0]} rows with a valid Feed')
    dfone["FIRST_YEAR"] = ""
    for f in filtered.ID:
        print (f'This is {f}')
        episode_list = dftwo.loc[dftwo['ID']==f]
        print(episode_list.shape)
        minimum_year = episode_list.EPISODE_YEAR.min().year
        maximum_year = episode_list.EPISODE_YEAR.max().year
        minimum_length = episode_list.EPISODE_TOTAL_MINUTES.min()
        maximum_length = episode_list.EPISODE_TOTAL_MINUTES.max()
        print(f'For this channel, first year is {minimum_year}. Last year is {maximum_year}')
        print(f'For this channel, shortest podcast is {minimum_length} minutes. Longest podcast is {maximum_length} minutes')        
        dfone.loc[dfone["ID"]==f,"FIRST_YEAR"] = minimum_year
        dfone.loc[dfone["ID"]==f,"LAST_YEAR"] = maximum_year 
        dfone.loc[dfone["ID"]==f,"MIN_PODCAST_LENGTH"] = minimum_length     
        dfone.loc[dfone["ID"]==f,"MAX_PODCAST_LENGTH"] = maximum_length        

In [94]:
defineYearRange(channels, episodes)

CHANNELS has 16 rows with a valid Feed
This is 519582740
(55, 7)
For this channel, first year is 2012. Last year is 2020
For this channel, shortest podcast is 2.0 minutes. Longest podcast is 55.0 minutes
This is 212828612
(50, 7)
For this channel, first year is 2018. Last year is 2020
For this channel, shortest podcast is 7.0 minutes. Longest podcast is 45.0 minutes
This is 79930757
(323, 7)
For this channel, first year is 2007. Last year is 2020
For this channel, shortest podcast is 1.0 minutes. Longest podcast is 44.0 minutes
This is 1153935086
(323, 7)
For this channel, first year is 2007. Last year is 2020
For this channel, shortest podcast is 1.0 minutes. Longest podcast is 44.0 minutes
This is 1470308447
(23, 7)
For this channel, first year is 2019. Last year is 2020
For this channel, shortest podcast is 20.0 minutes. Longest podcast is 84.0 minutes
This is 415333788
(49, 7)
For this channel, first year is 2008. Last year is 2016
For this channel, shortest podcast is 0.0 minutes.

In [95]:
channels

,ID,CHANNEL_NAME,EPISODE_COUNT,ITUNES_PREVIEW_URL,PUBLISHER_FEED,FIRST_YEAR,LAST_YEAR,MIN_PODCAST_LENGTH,MAX_PODCAST_LENGTH
0,519582740,Infectious Diseases Society of America,55,https://podcasts.apple.com/us/podcast/infectio...,https://idsocietypodcast.libsyn.com/rss,2012,2020.0,2.0,55.0
1,212828612,Emerging Infectious Diseases,50,https://podcasts.apple.com/us/podcast/emerging...,https://www2c.cdc.gov/podcasts/createrss.asp?t...,2018,2020.0,7.0,45.0
2,79930757,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml,2007,2020.0,1.0,44.0
3,1153935086,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml,2007,2020.0,1.0,44.0
4,1470308447,Breakpoints,23,https://podcasts.apple.com/us/podcast/breakpoi...,https://pinecast.com/feed/sidp,2019,2020.0,20.0,84.0
5,415333788,Infectious Disease,49,https://podcasts.apple.com/us/podcast/infectio...,http://wwwf.imperial.ac.uk/imedia/itunes_colle...,2008,2016.0,0.0,73.0
6,1368469324,The Open Forum Infectious Diseases Podcast,28,https://podcasts.apple.com/us/podcast/the-open...,https://feeds.blubrry.com/feeds/the_ofid.xml,2014,2020.0,NaN,NaN
7,621579678,Infectious Disease & Antibiotic Resistance,7,https://podcasts.apple.com/us/podcast/infectio...,None,,NaN,NaN,NaN
8,275853748,PeerView Infectious Diseases CME/CNE/CPE Audio...,16,https://podcasts.apple.com/us/podcast/peerview...,https://c.peerview.com/podcast/xml/infectiousd...,2018,2020.0,21.0,92.0
9,833385368,Infectious Disease Dynamics,53,https://podcasts.apple.com/us/podcast/infectio...,http://rss.sms.cam.ac.uk/rss/collection/153939...,2013,2014.0,NaN,NaN
